In [34]:
from util import read_csv
from wikipedia_crawler import get_famous_people_and_save
from create_dataframe import merge_DataFrames_and_save

import numpy as np
import pandas as pd

In [35]:
#Create Data if it does not exit
get_famous_people_and_save()
merge_DataFrames_and_save()

c:\Users\Pascal\Documents\FH Project\dawr\dawr-grisons\src\util.py:45: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead.
  df.replace('---', pd.np.nan, inplace=True)
c:\Users\Pascal\Documents\FH Project\dawr\dawr-grisons\src\util.py:45: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead.
  df.replace('---', pd.np.nan, inplace=True)


In [36]:
def is_prime(n) -> int:
    if n < 2:
        return 0
    for i in range(2, int(n**0.5) + 1):
        if n % i == 0:
            return 0
    return 1

In [37]:
df = read_csv('data', 'combined_data_set.csv')

c:\Users\Pascal\Documents\FH Project\dawr\dawr-grisons\src\util.py:45: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead.
  df.replace('---', pd.np.nan, inplace=True)


In [38]:
df['Bevoelkerungsdichte_normalized_inverse'] = 1 - (df['Bevoelkerungsdichte_pro_km2'] - df['Bevoelkerungsdichte_pro_km2'].min()) / (df['Bevoelkerungsdichte_pro_km2'].max() - df['Bevoelkerungsdichte_pro_km2'].min())
df['Auslaender_in_prozent_normalized_inverse'] = (df['Auslaender_in_prozent'].max() - df['Auslaender_in_prozent']) / (df['Auslaender_in_prozent'].max() - df['Auslaender_in_prozent'].min())
df['Buenzli_Rank_ranking_1'] = ((0.9 * df['Bevoelkerungsdichte_normalized_inverse'] + 1.6 * df['Auslaender_in_prozent_normalized_inverse'] + 0.5 * df['Wahlbeteiligung']) / 3) * 100

#x_normalized_inverse = 1 - (x - x_min) / (x_max - x_min)
#x_normalized = (x - x_min) / (x_max - x_min)

df['Bevoelkerungsdichte_normalized'] = (df['Bevoelkerungsdichte_pro_km2'] - df['Bevoelkerungsdichte_pro_km2'].min()) / (df['Bevoelkerungsdichte_pro_km2'].max() - df['Bevoelkerungsdichte_pro_km2'].min())
df['Einwohner_normalized'] = (df['Einwohner'] - df['Einwohner'].min()) / (df['Einwohner'].max() - df['Einwohner'].min())
df['Socialising_Rank_ranking_2'] = ((1.4 * df['Bevoelkerungsdichte_normalized'] + 1.1 * df['Einwohner_normalized'] + 0.5 * df['No_ratio']) / 3) * 100

# Sum up the famous people for all districts (2 criteria used)
# Now calculate a ratio of famous people per capita



cols_to_show = ['Gemeinde_Name', 'Buenzli_Rank_ranking_1', 'Socialising_Rank_ranking_2']
df = df.sort_values(by='Buenzli_Rank_ranking_1', ascending=False)
print(df[cols_to_show])

           Gemeinde_Name  Buenzli_Rank_ranking_1  Socialising_Rank_ranking_2
23            Tschappina               96.164215                    7.041975
73                 Furna               94.680090                   10.399434
25             Safiental               93.900044                    9.056225
76  Conters im Prättigau               93.880634                   10.443522
20               Flerden               93.273509                   11.450971
..                   ...                     ...                         ...
90             Landquart               47.609104                   48.428015
55            Silvaplana               47.442708                    9.262102
22                Thusis               41.596902                   23.339087
79                  Chur               39.491647                   88.629474
52            St. Moritz               32.450354                   22.000201

[101 rows x 3 columns]


In [39]:
famous_count_surses = df[df['Gemeinde_Name'] == 'Surses']['Famous_Person_count'].reset_index(drop=True)
famous_count_vaz = df[df['Gemeinde_Name'] == 'Vaz/Obervaz']['Famous_Person_count'].reset_index(drop=True)

print(famous_count_surses)
print(famous_count_vaz)

print (famous_count_vaz > famous_count_surses)


0    0.0
Name: Famous_Person_count, dtype: float64
0    10.0
Name: Famous_Person_count, dtype: float64
0    True
Name: Famous_Person_count, dtype: bool


In [40]:
# Ranking stuff
# create a sample dataframe
df = pd.DataFrame({'Name': ['Alice', 'Bob', 'Charlie', 'David'],
                   'GPA': [3.5, 3.2, 3.9, 3.6],
                   'Test Scores': [85, 90, 95, 80],
                   'Attendance': [90, 95, 80, 85],
                   'Participation': [75, 80, 85, 90]})
                   
# normalize the factors using min-max normalization
normalized = (df[['GPA', 'Test Scores', 'Attendance', 'Participation']] 
              - df[['GPA', 'Test Scores', 'Attendance', 'Participation']].min()) / (df[['GPA', 'Test Scores', 'Attendance', 'Participation']].max() 
                   - df[['GPA', 'Test Scores', 'Attendance', 'Participation']].min())

# combine the normalized factors into a single score
df['Ranking Score'] = normalized.mean(axis=1) * 100

In [41]:
# define the weights for each factor
weights = {'GPA': 0.3, 'Test Scores': 0.4, 'Attendance': 0.2, 'Participation': 0.1}

# multiply each column by its weight factor
weighted = df[['GPA', 'Test Scores', 'Attendance', 'Participation']].multiply(pd.Series(weights), axis=1)

# normalize the weighted factors using min-max normalization
normalized = (weighted - weighted.min()) / (weighted.max() - weighted.min())

# combine the normalized, weighted factors into a single score
df['Ranking Score'] = normalized.mean(axis=1) * 100

In [42]:
# define the weights for each factor
weights = {'GPA': 0.3, 'Test Scores': 0.4, 'Attendance': 0.2, 'Participation': 0.1}

# define a function to calculate conditional weights
def get_weight(value):
    if np.isnan(value) or value == 1:
        return 0
    elif value in [2, 3, 5, 7]:
        return 2
    else:
        return 1

# calculate the conditional weights for the GPA column
conditional_weights = df['GPA'].apply(get_weight)

# multiply the conditional weights by the weights for the GPA column
weighted_gpa = df['GPA'] * conditional_weights * weights['GPA']

# multiply the other columns by their weights
weighted_others = df[['Test Scores', 'Attendance', 'Participation']] * pd.Series(weights)[['Test Scores', 'Attendance', 'Participation']]

# combine the weighted factors into a single score
total_weighted = pd.concat([weighted_gpa, weighted_others], axis=1)
normalized = (total_weighted - total_weighted.min()) / (total_weighted.max() - total_weighted.min())
df['Ranking Score'] = normalized.mean(axis=1) * 100